# **Pressure Losses Notebook**

## **Objective**  
This notebook aims to validate the implementation of **pressure drop** models by comparing **singular** and **regular** pressure drops in both **hydraulic** and **multiphase** problems. The study is conducted using three numerical discretizations:  
- **VDF**  
- **PolyMAC_P0P1NC**  
- **PolyMAC_P0**  

## **Methodology**  
### 1. **Singular Pressure Drop (PDC singulière)**  
   - Applied on a **single face** inside the computational domain.  
   - Defined using a loss coefficient applied to a specific interface.  
   - The pressure drop is given by:  
     $$
     \Delta P = k \frac{1}{2} \rho U^2
     $$  
     where:  
     - $k$ is the **loss coefficient**,  
     - $\rho$ is the **fluid density**,  
     - $U$ is the **velocity of the flow**.  

### 2. **Regular Pressure Drop (PDC régulière)**  
   - Applied over **multiple cells** inside the domain.  
   - Defined through a **hydraulic diameter** $d_h$ and a **friction coefficient** $f$ over a subzone.  
   - The pressure drop follows the Darcy-Weisbach equation:  
     $$
     \Delta P = f \frac{L}{d_h} \frac{1}{2} \rho U^2
     $$  
     where:  
     - $f$ is the **friction coefficient**,  
     - $L$ is the **length of the pressure loss zone**,  
     - $d_h$ is the **hydraulic diameter**.  

## **Choice of Parameters for Consistency**  
To ensure that both models yield the same pressure drop, the following parameter choices are made:  
- $L = h_{\text{grid}}$  
- $f = k$  
- $d_h = h_{\text{grid}}$  

With these choices, the pressure drop in the regular case simplifies to:  
$$
\Delta P = k \frac{L}{h_{\text{grid}}} \frac{1}{2} \rho U^2
$$  
By setting $L = h_{\text{grid}}$, we obtain:  
$$
\Delta P = k \frac{1}{2} \rho U^2
$$  
which is identical to the singular pressure drop formulation.  

## **Variables Used**  
- **h_grid**: Represents the **hydraulic diameter** used in the regular pressure drop formulation.  
- **z_grid**: Defines the **position** of the grid in the domain where the pressure drop is applied.  
- **k**: Corresponds to the **loss coefficient**, which quantifies the resistance induced by the singular and regular pressure drops.  

## **Results and Observations**  
- The pressure at the **inlet and outlet** of the channel is observed to be **identical** for both methods, confirming their consistency.  

This study ensures the validity of the numerical implementation of pressure drop models across different discretizations and problem configurations.

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse & Elie Saikali")

### Description: 



In [ ]:
import os
run.reset()
run.initBuildDirectory()

list_dis = {"VDF" : ["jdd.data", "jdd_pb_multi.data"],
            "PolyMAC_P0P1NC" : ["jdd.data", "jdd_pb_multi.data"],
            "PolyMAC_P0" : ["jdd.data", "jdd_pb_multi.data"]}

h_grid = 0.3
z_grid = 0.5
k = 10.0

for dis, jdds in list_dis.items():
    for jdd in jdds:
        # singuliere
        pdc = f"perte_charge_singuliere {{ dir KY coeff {k} surface {{ y = {z_grid} grille orientation Champ_Uniforme 2 0 1 }} }}"
        run.addCaseFromTemplate(jdd, f"{dis}/case_Ks", {"zb" : z_grid - 0.1, "zt" : z_grid + 0.1,"pdc" : pdc, "dis" : dis})
        # reguliere
        pdc = f"perte_charge_isotrope {{ diam_hydr champ_uniforme 1 {h_grid} lambda {k} sous_zone grille }}"
        run.addCaseFromTemplate(jdd, f"{dis}/case_{h_grid:3.1f}", {"zb" : z_grid - 0.5 * h_grid, "zt" : z_grid + 0.5 * h_grid,"pdc" : pdc, "dis" : dis})

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

Graph = plot.Graph("Residuals : PDC Singulière vs Régulière", nY=2)
Graph.addPlot(0)
for i, h in enumerate([f"{h_grid:3.1f}", "Ks"]):
    for dis, jdds in list_dis.items():
        for jdd in jdds:
            l = "Ksing" if h == "Ks" else f"Kreg, h = {h}"
            if l == "Ksing":
                Graph.addResidu(f"{dis}/case_Ks/{jdd[:-5]}.dt_ev", label=f"{jdd[:-5]} {l}, {dis}")

Graph.addPlot(1)
for i, h in enumerate([f"{h_grid:3.1f}", "Ks"]):
    for dis, jdds in list_dis.items():
        for jdd in jdds:
            l = "Ksing" if h == "Ks" else f"Kreg, h = {h}"
            if l != "Ksing":
                Graph.addResidu(f"{dis}/case_{h}/{jdd[:-5]}.dt_ev", label=f"{jdd[:-5]} {l}, {dis}")           

Graph.scale(yscale="log")


In [ ]:
a = plot.Graph("Pression : PDC Singulière vs Régulière", nY=2)

a.addPlot(0)
x = [i / 100 for i in range(101)]
dp = [0.5 * 1000.0 * k if x_ < z_grid else 0.0 for x_ in x]
a.add(x, dp, label="Analytique", marker="o")
for i, h in enumerate([f"{h_grid:3.1f}", "Ks"]):
    for dis, jdds in list_dis.items():
        for jdd in jdds:
            l = "Ksing" if h == "Ks" else f"Kreg, h = {h}"
            if l == "Ksing":
                a.addSegment(f"{run.BUILD_DIRECTORY}/{dis}/case_{h}/{jdd[:-5]}_P.son", label=f"{jdd[:-5]} {l}, {dis}")
                a.label("z [m]", "pression")
                a.legend(loc="lower center", bbox_to_anchor=(0.5, -0.4))

a.addPlot(1)
x = [i / 100 for i in range(101)]
dp = [0.5 * 1000.0 * k if x_ < z_grid else 0.0 for x_ in x]
a.add(x, dp, label="Analytique", marker="o")
for i, h in enumerate([f"{h_grid:3.1f}", "Ks"]):
    for dis, jdds in list_dis.items():
        for jdd in jdds:
            l = "Ksing" if h == "Ks" else f"Kreg, h = {h}"
            if l != "Ksing":
                a.addSegment(f"{run.BUILD_DIRECTORY}/{dis}/case_{h}/{jdd[:-5]}_P.son", label=f"{jdd[:-5]} {l}, {dis}")
                a.label("z [m]", "pression")
                a.legend(loc="lower center", bbox_to_anchor=(0.5, -0.4))
